## Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

## Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing

In [5]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing.base import *
from robusta.pipeline import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            Imputer(strategy="most_frequent"),
            LabelEncoder(),
            ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,gr_Age,gr_Fare,gr_SibSp,gr_Parch,le_Pclass,le_Sex,le_Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


## Fold Preparation

In [ ]:
from sklearn.model_selection import KFold
from robusta.resampler import *

encoder = FeatureUnion([
    ('category', make_pipeline(
        TypeSelector(['category', 'object']), 
        TargetEncoderCV(cv=4).set_params(encoder__smoothing=200.0),
    )),
    ('numeric', make_pipeline(
        TypeSelector(np.number),
        Identity(),
    )),
])

resampler = SMOTE(random_state=100, k_neighbors=200)

fold_pipe = make_pipeline(resampler, encoder)

F_train = fold_pipe.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

## Model

In [6]:
%%time
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

CPU times: user 378 ms, sys: 10.7 ms, total: 388 ms
Wall time: 67.6 ms


/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [7]:
y = y_train.copy()
y[y_train == 1] = '+'
y[y_train == 0] = '-'

y.head()

PassengerId
1    -
2    +
3    +
4    +
5    -
Name: Survived, dtype: object

In [11]:
le = LabelEncoder1D()
yt = le.fit_transform(y)
le.inverse_transform(yt).equals(y)

True

In [12]:
cols = X_train.columns

le = LabelEncoder1D()
colst = le.fit_transform(cols)
le.inverse_transform(colst).equals(cols)

True

In [14]:
%%time
from robusta.crossval._crossval import cross_val_stack

results = cross_val_stack(model, X_train, y, None, 5, X_test, method='predict_proba')

PassengerId
1      -
2      +
3      +
4      +
5      -
6      -
7      -
8      -
9      +
10     +
11     +
12     +
13     -
14     -
15     -
16     +
17     -
18     +
19     -
20     +
21     -
22     +
23     +
24     +
25     -
26     +
27     -
28     -
29     +
30     -
      ..
862    -
863    +
864    -
865    -
866    +
867    +
868    -
869    -
870    +
871    -
872    +
873    -
874    -
875    +
876    +
877    -
878    -
879    -
880    +
881    +
882    -
883    -
884    -
885    -
886    -
887    -
888    +
889    -
890    +
891    -
Name: Survived, Length: 891, dtype: object
PassengerId
1      -
2      +
3      +
4      +
5      -
6      -
7      -
8      -
9      +
10     +
11     +
12     +
13     -
14     -
15     -
16     +
17     -
18     +
19     -
20     +
21     -
22     +
23     +
24     +
25     -
26     +
27     -
28     -
29     +
30     -
      ..
862    -
863    +
864    -
865    -
866    +
867    +
868    -
869    -
870    +
871    -
872    +
873   

In [8]:
results

((array([[0.90800585, 0.09199415],
         [0.00458404, 0.99541596],
         [0.7134699 , 0.2865301 ],
         [0.00153002, 0.99846998],
         [0.96390994, 0.03609006],
         [0.80976157, 0.19023843],
         [0.77922425, 0.22077575],
         [0.5068566 , 0.4931434 ],
         [0.53658125, 0.46341875],
         [0.02303346, 0.97696654],
         [0.1312732 , 0.8687268 ],
         [0.27227806, 0.72772194],
         [0.7384917 , 0.2615083 ],
         [0.96182598, 0.03817402],
         [0.14747425, 0.85252575],
         [0.30551785, 0.69448215],
         [0.90715141, 0.09284859],
         [0.96176958, 0.03823042],
         [0.0532653 , 0.9467347 ],
         [0.35716835, 0.64283165],
         [0.78485636, 0.21514364],
         [0.96275354, 0.03724646],
         [0.12430768, 0.87569232],
         [0.96523579, 0.03476421],
         [0.82186357, 0.17813643],
         [0.96002345, 0.03997655],
         [0.9837626 , 0.0162374 ],
         [0.90184596, 0.09815404],
         [0.12678595

In [7]:
from sklearn.model_selection import cross_val_score

cv = 5
scoring = 'accuracy'

estimator = make_pipeline(model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(resampler, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [encoder + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(encoder, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [resampler + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(resampler, encoder, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [resampler + encoder + model]'.format(np.mean(scores), np.std(scores)))

estimator = make_pipeline(encoder, resampler, model)
scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)
print('{:.4f} ± {:.4f} [encoder + resampler + model]'.format(np.mean(scores), np.std(scores)))

# 0.8216 ± 0.0370 (model)
# 0.8249 ± 0.0320 (encoder + model)
# 0.8261 ± 0.0336 (encoder + model)

0.8340 ± 0.0263 [model]
0.8261 ± 0.0248 [encoder + model]
0.8306 ± 0.0212 [resampler + model]
0.8306 ± 0.0220 [resampler + encoder + model]
0.8103 ± 0.0077 [encoder + resampler + model]


In [8]:
#estimator = make_pipeline(encoder, model)
estimator = make_pipeline(model)

## Submit (Out-of-Fold & Test Predictions)

In [121]:
%%time
from robusta.crossval import stacking, StackingTransformer

S_train, S_test = stacking([model], X_train, y_train, X_test, n_folds=cv, 
                           stratified=True, regression=False)

#stack = StackingTransformer([('te+lgbm', estimator)], n_folds=cv, stratified=True, regression=False)
#S_train = stack.fit_transform(X_train, y_train)
#S_test = stack.transform(X_test)
#S_train, S_test = stacking([estimator], X_train, y_train, X_test, n_folds=cv, 
#                           stratified=True, regression=False)

CPU times: user 2.39 s, sys: 623 ms, total: 3.01 s
Wall time: 394 ms


In [134]:
y_oof = pd.Series(S_train[:,0], index=X_train.index, name=target)
y_sub = pd.Series(S_test[:,0], index=X_test.index, name=target)

y_sub.head()

PassengerId
892    0
893    0
894    0
895    1
896    0
Name: Survived, dtype: int64

In [52]:
import os

path = os.path.join(os.getcwd(), 'pred')
if not os.path.exists(path):
    os.mkdir(path)

sub_path = os.path.join(path, '0 sub baseline.csv')
oof_path = os.path.join(path, '0 oof baseline.csv')

y_sub.to_csv(sub_path, header=True)
y_oof.to_csv(oof_path, header=True)

### Score:
### `[CV] 0.8249 ± 0.0314`
### `[LB] 0.7656`